In [1]:
import pandas as pd
import json
from functools import reduce
import numpy as np
import os

In [2]:
cwd = os.getcwd()
results_dir = f"{cwd}/results"
saving_dir = f"{cwd}/results/dataset_tabular_merged.csv"
all_df_list = []
for type_ in ["", "_typical"]:
    df = pd.read_csv(f"results/dataset_tabular{type_}.csv")
    emotions = pd.read_csv(f"results/emotions{type_}.csv")
    sentiments = pd.read_csv(f'results/sentiments{type_}.csv')
    complexities = pd.read_csv(f'results/complexities{type_}.csv')
    entities = json.load(open(f'results/entities{type_}.csv'))
    imagery = json.load(open(f'results/imagery_words_proportion{type_}.csv'))
    stopwords = json.load(open(f'results/stopwords_proportion{type_}.csv'))
    mean_lengths = json.load(open(f'results/mean_sentence_lengths{type_}.csv'))

    other_scores = {
        "Unnamed: 0": list(range(0, 77)),
        "entities_proportion_in_speech": entities["proportion_in_speech"],
        "imagery_proportion_in_speech": imagery["proportion_in_speech"],
        "stopwords_proportion_in_speech": stopwords["proportion_in_speech"],
        "mean_sentence_lengths": mean_lengths["mean"],
        "label": np.ones(77).tolist() if type_ == '' else np.zeros(77).tolist()

    }
    df_other_scores = pd.DataFrame(other_scores)

    dfs = [df, emotions, sentiments, complexities, df_other_scores]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Unnamed: 0']), dfs)
    all_df_list.append(df_merged)

In [3]:
df_all = pd.DataFrame.append(all_df_list[0], all_df_list[1])
df_all = pd.DataFrame.reset_index(df_all).iloc[:, 2:]

/tmp/ipykernel_28265/48323824.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = pd.DataFrame.append(all_df_list[0], all_df_list[1])


In [4]:
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
df_all.to_csv(saving_dir)